# utils data2

In [11]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

import requests
import json
import sys
import os
import time
import operator
from types import *
from collections import Counter
from datetime import datetime


In [5]:
DESC = "FLALL2"

LOG = "../../LOG.txt"
LOGDIR     = "../../dmodels/"
LOGDAT     = "../../data/"
DSJ        = "/data_json.txt"
DSC        = "/datasc.csv"   
DC         = "/datac.csv"
DL         = "/datal.csv"

print("set desc!")

COL_DS     = LOGDAT + DESC + DC 
ALL_DSJ    = LOGDAT + DESC + DSJ 
ALL_DS     = LOGDAT + DESC + DSC 
pp_abs     = False

set desc!


## Functions

In [12]:
#FD
def feed_data(dataJJ, d_st = False, pand=False, p_col = False,  p_all = True, p_down=False):
    #index_col=0 if p_abs else 2 #abs=F => 2 == 6D
    index_col = 2 #name - num, abs = cc
    # col_df = pd.read_csv(COL_DS, index_col=index_col, sep=',', usecols=[0,1,2,3])    
    col_df = pd.read_csv(COL_DS, index_col=index_col, sep=',', usecols=[0,1,2,3])    
    col_df = col_df.fillna(0)
    print("input-no={}".format( len(col_df )))
    
    try:
        indx = dst.columns
        indx = indx.delete(2)
    except NameError:  #TypeError
        indx = col_df.index
        indx = indx.insert(0, "M")
        indx = indx.insert(1, "FP")  # indx = indx.insert(2, "FP_P")
    print(type(indx[5]));print(indx[5]);
    if p_col:    
        dataTest_label = []
        dataJJ = "["
        for i in range(len(col_df)): 
            #fpp = cc( int(  col_df.iloc[i]["fp"]  ))
            fpp = int(  col_df.iloc[i]["fp"]  )
            dataTest_label.append(  fpp ) 
            #dataJJ += '{"m":"'+str(i)+'",'+'"'+str(col_df.iloc[i].name)+'"'+":1},"
            dataJJ += '{"m":"'+str(col_df.iloc[i].name)+'",'+'"'+str(col_df.iloc[i].name)+'"'+":1},"
        dataJJ += '{"m":"0"}]';  dataTest_label.append(0)
        # dataJJ += ']'
        dataJJ = json.loads(dataJJ)

    if pd.core.indexes.numeric.is_integer_dtype(col_df.index):  isInt = True
    else: isInt = False

    json_df  = pd.DataFrame(columns=indx); df_entry = pd.Series(index=indx)
    
    df_entry = df_entry.fillna(0) 
    ccount = Counter()

    if(isinstance(dataJJ, list)):json_data = dataJJ
    else: json_str=open(dataJJ).read();  json_data = json.loads(json_str)
    
    if p_all: ll = range(len(json_data))
    else: ll = range(ll_st, ll_en)
    # pp_abs = True  # test 
    print("PP_ABS = " + str(pp_abs));    
    #for i in range(2):
    for i in ll: # print(i)
        df_entry *= 0
        m = str(json_data[i]["m"])
        df_entry.name = m
        df_entry["M"] = m
        for key in json_data[i]:
            if key != "m": 
                # key_wz = key if pp_abs else int(key)  #str(int(key)) FRFLO - int // FRALL str!
                
                if isInt : key_wz = int(key)            # if comp NOT conatin letters
                else: key_wz = str(key) #str(key)       # if comp contains letters
                # print(type(key))                
                try: #filling of key - experimental or COMP 
                    if d_st:
                        ds_comp = col_df.loc[key_wz] #print(ds_comp) # THIS IS THE MOST TIME CONSUMING OP. 
                        col_key = ds_comp.cc if pp_abs else  str(ds_comp.name) #
                    else: 
                        col_key = int(key) #str(int(key)) 
                    
                    col_key = int(col_key) # or int... 
                    #if isInt : col_key = int(col_key)     # if comp NOT conatin letters
                    #else: key_wz = col_key = str(col_key)   

                    df_entry[col_key] =  np.float32(json_data[i][key]) # df_entry.loc[col_key]
                    # print(col_key); print(type(col_key))
                except: 
                    if d_st: print("m:{}-c:{} not included" .format(m, key_wz)); ccount[key_wz] +=1
        # ONLY USED TO CONVERT JSON TO EXCEL => EXCEL WILL BE SMALLER
        if p_down: df_entry = df_entry.replace(0, np.nan) # DANGER !!! 

        json_df = json_df.append(df_entry,ignore_index=False)
        if i % 1000 == 0: print("cycle: {}".format(i))
    print("Counter of comp. not included :"); print(ccount) # print(len(ccount))
 
        
    if p_col:  
        json_df["FP"] = dataTest_label
    
    if pand:  return json_df
    else:     
        if p_col: return json_df.iloc[:,3:].as_matrix().tolist(), dataTest_label
        else:     return json_df.iloc[:,3:].as_matrix().tolist() 


In [ ]:
def get_tests(url_test='url', force=False, pp_excel=False, pDataFile = "data_jsonX.txt", pLabelFile = "datalX.csv", p_dst=True, p_all = True ): 
    global dsp, flag_dsp 
    print(pp_excel)
    if flag_dsp or force: 
        flag_dsp = False
        # 2 -- READ EXCEL 
        if pp_excel: dsp = pd.read_csv( tf.gfile.Open( LOGDAT + DESC + "/datasc_tx.csv"  ), sep=None, skipinitialspace=True,  engine="python" )
        else: # 1 -- READ JSON 
            if url_test != 'url':           # test  file 
                json_data = url_test + pDataFile
                tmpLab = pd.read_csv( url_test + pLabelFile, sep=',', usecols=[0,1])    
                tmpLab = tmpLab.loc[:,'fp'].tolist()
                #DESC     = "FREXP1_X"
            else:                           # get data test JSON = url
                json_str, tmpLab = get_data_test(1) 
                json_data = json.loads(json_str)
                #DESC =  'matnrList...'
        
            dsp = feed_data(json_data ,pand=True, d_st=p_dst, p_all = p_all)       #d_st = display status
            
            if p_all: dsp["FP"] = tmpLab      #normalize(2)  # del dsp['FP_P']
        # if p_all: dsp.insert(2, 'FP_P', dsp['FP'].map(lambda x: cc( x )))  

    else:    
        return True

In [10]:
def testsJ2(pDesc, excel=True, split = False, pTest = True):
    start = time.time()
    print("___JSON!___" +  datetime.now().strftime('%H:%M:%S')  )

    # url_test = LOGDAT + "FREXP1/" ; dataFile = "data_jsonX.txt";  labelFile = "datalX.csv" ;   #url_test = "url"
    # setDESC("FLALL2"); url_test = LOGDAT + "FLALL2/" ; dataFile = "frall2_json.txt"; labelFile = "datal.csv" 
    url_test = LOGDAT + pDesc + "/" ; dataFile = "frall2_json.txt"; labelFile = "datal.csv" 
    
    if pTest:                                               #   disp   all
        get_tests(url_test, False, False, dataFile, labelFile, False, False ); tmp = dsp
    else: 
        pass
        #get_columns(False, False, True); tmp = dsc
    
    # del tmp['FP_P']

    #if split: pass # separate betweent TR and EV     

    print("data read - time:{}" .format(float(time.time() - start) ))
    down_excel(tmp,  excel)

In [9]:
def down_excel(data, excel_flag): 
    if excel_flag: 
        writer = pd.ExcelWriter(LOGDAT+'json2excel.xlsx')
        data.to_excel(writer, sheet_name='Sheet1')
        writer.save()
        print("JSON downloaded into excel! ")

In [7]:
def main1():    # convert json2excel_base
    print("hi1")
    # md.mainRead2(ALL_DS, 1, 2, all = True, shuffle = True  ) 
    #pDesc = "FLALL2"
    #setDESC(pDesc)
    # mainRead2(ALL_DS, 1, 2, all = False ) # For testing I am forced to used JSON - column names and order may be different! 
    testsJ2(pDesc = pDesc, excel=True, split = False, pTest = False)

In [8]:
# mainRead2(ALL_DS, 1, 2, all = True, shuffle = False ) 
main1()

hi1


NameError: name 'testsJ2' is not defined

# Testing

In [14]:
index_col = 2
tol_df = pd.read_csv(COL_DS, index_col=index_col, sep=',', usecols=[0,1,2,3])    


In [18]:
type(tol_df.iloc[0]["fp"]) 

str